In [2]:
import numpy as np 
import os 
import sys 
import glob 
import xarray as xr
import dask
import time
from dask.distributed import Client

path = "/home/b/b382291/git/one_pass"
sys.path.append(path)
os.chdir(path)

from one_pass.convert_time import convert_time
from one_pass.check_request import check_request
from one_pass import util
from one_pass.opa import Opa
!git branch 

### select input and out put directory
outdir = '/work/bb1153/b382220/KOSTRA/Tests/iamser_out/'
# Check if the directory exists, and create it if not
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
indir = "/work/bb1153/b382220/KOSTRA/kostrax/DWData/"
file_list = sorted([file for file in os.listdir(indir) if file.endswith('.nc')])

!hostname

### load data and select variables
# ds = xr.open_mfdataset(f'{indir}*.nc') ### load data as dask array
# pr = ds['pr'] ### select data variable for total precipitation
# da = xr.DataArray(pr)
# # have to change the precision, data set has strange time stamps
# da['time'] = da.time.astype("datetime64[s]")

  iams
  main
* output_for_bias_corr
  t_digest
  test_package
l40052.lvt.dkrz.de


extracting a small subset of the data otherwise it's crazy long 

In [3]:
file_path = "/home/b/b382291/git/one_pass/tests/pr_12_months.nc"
data = xr.load_dataset(file_path)
da = data
da = da.pr# [:,0:10,0:10]


In [4]:

# this is our user requst 

pass_dic = {"stat" : "iams",
"percentile_list" : None,
"thresh_exceed" : None,
"stat_freq": "annually",
"output_freq": "annually",
"time_step": 60,
"variable": "pr",
"save": True,
"checkpoint": True,
"checkpoint_filepath": "/scratch/b/b382291/data/",
"out_filepath": "/scratch/b/b382291/data"}

start = 0
stop =  365*24 # end of 2071
step = 200# don't go above this for now 

opa_stat = Opa(pass_dic)

for i in range(start, stop, step): 

    data = da.isel(time=slice(i,i+step)) # extract moving window 'simulating streaming'
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")

    dm = opa_stat.compute(data)

start_date: 2071-01-01T00:30:00.000000000 stop_date: 2071-01-09T07:30:00.000000000


start_date: 2071-01-09T08:30:00.000000000 stop_date: 2071-01-17T15:30:00.000000000
start_date: 2071-01-17T16:30:00.000000000 stop_date: 2071-01-25T23:30:00.000000000
start_date: 2071-01-26T00:30:00.000000000 stop_date: 2071-02-03T07:30:00.000000000
start_date: 2071-02-03T08:30:00.000000000 stop_date: 2071-02-11T15:30:00.000000000
start_date: 2071-02-11T16:30:00.000000000 stop_date: 2071-02-19T23:30:00.000000000
start_date: 2071-02-20T00:30:00.000000000 stop_date: 2071-02-28T07:30:00.000000000
start_date: 2071-02-28T08:30:00.000000000 stop_date: 2071-03-08T15:30:00.000000000
start_date: 2071-03-08T16:30:00.000000000 stop_date: 2071-03-16T23:30:00.000000000
start_date: 2071-03-17T00:30:00.000000000 stop_date: 2071-03-25T07:30:00.000000000
start_date: 2071-03-25T08:30:00.000000000 stop_date: 2071-04-02T15:30:00.000000000
start_date: 2071-04-02T16:30:00.000000000 stop_date: 2071-04-10T23:30:00.000000000
start_date: 2071-04-11T00:30:00.000000000 stop_date: 2071-04-19T07:30:00.000000000
star

In [3]:
np.shape(opa_stat.iams_cum)

(15, 100, 100)

In [3]:
### all functions
############################################################## to select available duration depending on the input data 
############################################################## time_unit(str) = 'minutes' or 'hours'; time_step(int) = e.g. 1 for hourly, 10 for ten minutes

#-- automatically check timestep of files and rescale
#-- remove durations that are lower than timestep and round
#-- check if duration is a fraction of multiple timesteps
def duration_pick(time_unit, time_step):
    durations  = [10080] ### list of all duration windows that may be passed on
    
    if time_unit != 'minutes': ### format time unit, when not minutes
        durations = [d/60 for d in durations]

    durations = [d for d in durations if d >= time_step] ### loop over duration windows that fall in the range of the time step
    for d in durations:
        if d%time_step != 0: ### only select natural numbers
            durations.remove(d)  
    durations = list(map(int, durations)) ### create list of only intergers
    return durations

############################################################## compute intensity annual maxima series (iams) for single duration window
############################################################## 

def dm_roller(data, duration): 

    ### compute rolling sum & select maximum
    rolling_sum = data.rolling(time=duration, center=True).sum().dropna('time')
    rolling_sum_max = rolling_sum.expand_dims(duration = ([duration])) 
        
    return rolling_sum_max

############################################################## For data of several years: compute intensity annual maxima series (iams) for single duration window
##############################################################

def iamser(data, years, durations):###(ds.'precipitation', list()selected_duration windows, list()years)
    with dask.config.set(**{'array.slicing.split_large_chunks': True}): ### set dask to slice large chunks 'False' also works dunno

        dmax_list = [] ### list to add each duartion sum max
        for durs in durations: ### loop over selected durations
            dmax = dm_roller(data, durs) ### call iamser function for one year
            dmax_list.append(dmax) ### append iams output to iams_list
        iamsfile = xr.concat(dmax_list, dim='duration') ### concat all durations into one array

        return iamsfile 


In [4]:
### pick possible duration windows
time_step = 1 ### define data temporal fruequency
time_unit = 'hour' ### define data time unit
sel_durs = duration_pick('hours',1) ### call sel_durs to select all possible windows within given time frquency & unit

years= [2071] # list(np.unique(pr.time.dt.year)) ### create list for all years in the given data set 

### run functions to create intensity annual  maxima series
iams = iamser(da, years, sel_durs)

In [5]:
iams = np.swapaxes(iams, 1, 2)
iams = np.swapaxes(iams, 2, 3)
iams

<xarray.DataArray 'pr' (duration: 1, time: 10, lat: 10, lon: 8593)>
array([[[[0.00595191, 0.00595191, 0.00595191, ..., 0.00847438,
          0.00846888, 0.00846642],
         [0.00638428, 0.00638428, 0.00638428, ..., 0.00845196,
          0.00844539, 0.00844318],
         [0.00696691, 0.00696691, 0.00696691, ..., 0.00839409,
          0.00838764, 0.00838594],
         ...,
         [0.00618551, 0.00618552, 0.00618552, ..., 0.00517988,
          0.00517881, 0.00517875],
         [0.00633201, 0.00633202, 0.00633202, ..., 0.00468379,
          0.0046829 , 0.00468286],
         [0.00673369, 0.00673371, 0.00673371, ..., 0.00434068,
          0.00433988, 0.00433983]],

        [[0.00596169, 0.00596169, 0.00596169, ..., 0.00849983,
          0.0084973 , 0.00849611],
         [0.00622426, 0.00622426, 0.00622426, ..., 0.00854829,
          0.00854596, 0.00854485],
         [0.00656363, 0.00656363, 0.00656363, ..., 0.00857953,
          0.00857743, 0.00857651],
...
         [0.02197132, 0.02197132, 0.02197135, ..., 0.01378065,
          0.01377666, 0.01377459],
         [0.02179365, 0.02179365, 0.02179366, ..., 0.01378725,
          0.01378107, 0.01377764],
         [0.02088523, 0.02088523, 0.02088524, ..., 0.01327786,
          0.01326827, 0.01326229]],

        [[0.01020392, 0.01020392, 0.01020392, ..., 0.01291295,
          0.01291094, 0.01290985],
         [0.01333416, 0.01333417, 0.01333417, ..., 0.01497668,
          0.01497387, 0.01497217],
         [0.01647928, 0.01647928, 0.01647932, ..., 0.01533155,
          0.01532782, 0.01532529],
         ...,
         [0.02396401, 0.02396401, 0.02396406, ..., 0.01387596,
          0.01387154, 0.01386911],
         [0.02401542, 0.02401542, 0.02401547, ..., 0.01409856,
          0.01409147, 0.01408762],
         [0.02298454, 0.02298454, 0.0229846 , ..., 0.01362234,
          0.01361001, 0.01360304]]]], dtype=float32)
Coordinates:
  * duration  (duration) int64 168
  * time      (time) datetime64[ns] 2071-01-04T12:30:00 ... 2071-12-28T12:30:00
  * lon       (lon) float64 -4.794 -4.766 -4.739 -4.711 ... -4.601 -4.574 -4.546
  * lat       (lat) float64 1.004 1.031 1.059 1.086 ... 1.169 1.196 1.224 1.251

In [6]:
one_pass = opa_stat.iams_cum[0,:, :,0:8593] 
two_pass = iams[0,:]

In [15]:
for i in range(1,11):
    print(two_pass.values[5,5,i].astype('float32'))


0.015380896
0.0153808985
0.015381386
0.0155489305
0.015776923
0.015947897
0.016339151
0.016920375
0.01734516
0.017557932


In [12]:
for i in range(10):
    print(one_pass[5,5,i].astype('float32'))

0.015380897
0.015380899
0.0153813865
0.015548931
0.015776923
0.015947897
0.016339151
0.016920375
0.01734516
0.017557932


In [20]:
one_pass[4,4,-1]

0.0

In [19]:
dp = 0.000001
np.allclose(two_pass[:,:,1:], one_pass[:,:,0:-1], rtol = dp, atol = dp)
  

True

comparison with two pass 

In [5]:
### all functions
############################################################## to select available duration depending on the input data 
############################################################## time_unit(str) = 'minutes' or 'hours'; time_step(int) = e.g. 1 for hourly, 10 for ten minutes

#-- automatically check timestep of files and rescale
#-- remove durations that are lower than timestep and round
#-- check if duration is a fraction of multiple timesteps
def duration_pick(time_unit, time_step):
    durations  = [5,10,15,20,30,45,60,90,120,180,240,360,540,720,1080,1440,2880,4320,5760,7200,8640, 10080] ### list of all duration windows that may be passed on
    
    if time_unit != 'minutes': ### format time unit, when not minutes
        durations = [d/60 for d in durations]

    durations = [d for d in durations if d >= time_step] ### loop over duration windows that fall in the range of the time step
    for d in durations:
        if d%time_step != 0: ### only select natural numbers
            durations.remove(d)  
    durations = list(map(int, durations)) ### create list of only intergers
    return durations

############################################################## compute intensity annual maxima series (iams) for single duration window
############################################################## 

def dm_roller(data, duration): 

    ### compute rolling sum & select maximum
    rolling_sum = data.rolling(time=duration, center=True).sum() 
    rolling_sum_max = rolling_sum.max(dim='time', skipna=True) 
    rolling_sum_max = rolling_sum_max.expand_dims(duration = ([duration])) 
        
    return rolling_sum_max

############################################################## For data of several years: compute intensity annual maxima series (iams) for single duration window
##############################################################

def iamser(data, years, durations):###(ds.'precipitation', list()selected_duration windows, list()years)
    with dask.config.set(**{'array.slicing.split_large_chunks': True}): ### set dask to slice large chunks 'False' also works dunno
        # Your array indexing/slicing operations
        all_years_iams = [] ### create list to gather yearly arrays
        #data_grouped = data.groupby('time.year') ### group data into years
        #years= list(np.unique(pr.time.dt.year)) ### create list with all years

        #group_with_time = group.assign_coords(time=group['time'])
        dmax_list = [] ### list to add each duartion sum max
        for durs in durations: ### loop over selected durations
            dmax = dm_roller(data, durs) ### call iamser function for one year
            dmax_list.append(dmax) ### append iams output to iams_list
        iamsfile = xr.concat(dmax_list, dim='duration') ### concat all durations into one array

        return iamsfile 


In [6]:
### pick possible duration windows
time_step = 1 ### define data temporal fruequency
time_unit = 'hour' ### define data time unit
sel_durs = duration_pick('hours',1) ### call sel_durs to select all possible windows within given time frquency & unit

### load data and select variables
#ds = xr.open_mfdataset(f'{indir}*.nc') ### load data as dask array
#ds = xr.open_mfdataset(f'{indir}{file_list[4]}')
#pr = ds['pr'] ### select data variable for total precipitationpr
#pr= pr[:,0:10,0:10]
#pr = ds['tp']
years= [2071] # list(np.unique(pr.time.dt.year)) ### create list for all years in the given data set 

### run functions to create intensity annual  maxima series
iams = iamser(da, years, sel_durs)

In [8]:
num = 15
np.allclose(iams.values[0:num,:,:], dm.pr.values[0,0:num,:,:], rtol = 0.000001, atol = 0.000001)

True